In [ ]:
import pandas as _hex_pandas
import datetime as _hex_datetime
import json as _hex_json

In [ ]:
hex_scheduled = _hex_json.loads("false")

In [ ]:
hex_user_email = _hex_json.loads("\"example-user@example.com\"")

In [ ]:
hex_run_context = _hex_json.loads("\"logic\"")

# 2021 EOY Fundraising Report



## Overview

This report summarizes our Actblue data for the 2021 fiscal year across all four of our committees. There was a slight decrease in total amount raised, total number of contributions, and average donation amount in 2021 compared to 2020. 

In [ ]:
# import jinja2
# raw_query = """
#     with donations as (
#       select * from  sunrise_dbt.donations
#     ),
#     
#     all_donations as (
#       select
#         donation_id,
#         amount,
#         date_trunc('month', paid_at) as month
#       from donations
#     ),
#     
#     donations_month as (
#       select
#         month,
#         sum(amount) as total_amount,
#         count(donation_id) as total_contributions,
#         total_amount / total_contributions as avg_amount
#       from all_donations
#       where month like '%2021%'
#       group by 1
#       order by 1
#     )
#     
#     select * from donations_month
# """
# sql_query = jinja2.Template(raw_query).render(vars())

In [ ]:
# import jinja2
# raw_query = """
#     with donations as (
#       select * from  sunrise_dbt.donations
#     ),
#     
#     all_donations as (
#       select
#         donation_id,
#         amount,
#         date_part_year(paid_at::date) as year
#       from donations
#     ),
#     
#     donations_year as (
#       select
#         year,
#         sum(amount) as total_amount,
#         count(donation_id) as total_contributions,
#         total_amount / total_contributions as avg_amount
#       from all_donations
#       where year not like '%2022%'
#       group by 1
#       order by 1
#     )
#     
#     select * from donations_year
# """
# sql_query = jinja2.Template(raw_query).render(vars())

In [ ]:
# import jinja2
# raw_query = """
#     with
#     
#     donations as (
#       select * from sunrise_dbt.donations
#     ),
#     
#     all_donations as (
#         select  
#          donation_id,
#          committee_name,
#          committee_id,
#          amount,
#          date_part_year(paid_at::date) as year
#         from donations
#       ),
#     
#     fundraising_summary_2020 as (
#         select
#          committee_id,
#          committee_name,
#          count(donation_id) as num_contributions_2020,
#          sum(amount) as total_amount_2020
#         from all_donations
#         where year = '2020'
#         group by 1,2
#       ),
#     
#     fundraising_summary_2021 as (
#         select
#          committee_id,
#          committee_name,
#          count(donation_id) as num_contributions_2021,
#          sum(amount) as total_amount_2021
#         from all_donations
#         where year = '2021'
#         group by 1,2
#       ),
#     
#       annual_donation_breakdown as (
#         select
#          fundraising_summary_2021.committee_id,
#          fundraising_summary_2021.committee_name,
#          num_contributions_2020,
#          num_contributions_2021,
#          total_amount_2020,
#          total_amount_2021
#         from fundraising_summary_2021
#         left join fundraising_summary_2020 using(committee_id)
#         order by 2
#       )
#     
#     select 
#         committee_name,
#         total_amount_2020,
#         total_amount_2021,
#         total_amount_2021 - total_amount_2020 as amount_difference,
#         amount_difference/total_amount_2020 as percent_difference
#     from annual_donation_breakdown
# """
# sql_query = jinja2.Template(raw_query).render(vars())

In [ ]:
# import jinja2
# raw_query = """
#     with summary as (
#         select
#           date_part_year(paid_at::date) as year,
#           sum(amount) as total_raised,
#           count(donation_id) as num_contributions,
#           count(distinct email) as total_donors,
#           avg(amount) as avg_donation,
#           (num_contributions::decimal / total_donors::decimal)::float as avg_contributions
#         from sunrise_dbt.donations
#         where year in (2020,2021)
#         group by 1
#         order by 1
#       )
#     
#       select * from summary
# """
# sql_query = jinja2.Template(raw_query).render(vars())

In [ ]:
total_2021 = round(donation_year["total_amount"].iloc[4], 2)
total_2020 = round(donation_year["total_amount"].iloc[3], 2)
total_difference = round(total_2020 - total_2021, 2)
contributions_2021 = donation_year["total_contributions"].iloc[4]

pct_month = round((donation_month["total_amount"].iloc[11] / total_2021) * 100, 2)

In [ ]:
total_2021

In [ ]:
total_2020

### Basic Stats

We raised a total of \${{total_2021}} and had over {{contributions_2021}} contributions through Actblue in 2021 across all our committees (Table 1). 

This is a decrease of \${{total_difference}} (-4.18%) compared to 2020 (Graph 1). In Table 2, we see that the decrease can be attributed to the decline in donations to the PAC committees. However, there was an increase in both the C3 and the C4 committees compared to last year.

In [ ]:
donation_summary

In [ ]:
import altair
chart_donation_year = altair.Chart.from_json("""
{
    "width": 500,
    "height": 500,
    "layer": [
        {
            "data": {
                "name": "layer00"
            },
            "mark": {
                "clip": true,
                "type": "bar",
                "tooltip": true
            },
            "encoding": {
                "x": {
                    "type": "ordinal",
                    "field": "year",
                    "title": "Year"
                },
                "y": {
                    "type": "quantitative",
                    "field": "total_amount",
                    "title": "Total Amount ($)"
                },
                "color": {
                    "value": "#D99E0B"
                }
            }
        }
    ],
    "$schema": "https://vega.github.io/schema/vega-lite/v4.json",
    "resolve": {
        "scale": {
            "y": "independent"
        }
    },
    "datasets": {
        "layer00": [
            {
                "name": "dummy",
                "value": 0
            }
        ]
    }
}
""")
chart_donation_year.datasets.layer00 = donation_year.to_json(orient='records')
chart_donation_year.display(actions=False)

In [ ]:
committees

In [ ]:
# import jinja2
# raw_query = """
#     with donations as (
#       select * from  sunrise_dbt.donations
#     ),
#     
#     quarterly_fundraising_summary as (
#         select
#           date_trunc('quarter', paid_at::date) as quarter_date_at,
#           date_part(quarter, paid_at::date) as quarter,
#           sum(amount) as total_amount,
#           count(distinct email) as total_donors,
#           count(donation_id) as num_contributions,
#           avg(amount) as avg_donation,
#           (num_contributions::decimal / total_donors::decimal)::float as avg_contributions
#         from donations
#         where quarter_date_at like '%2021%'
#         group by 1,2
#         order by 2
#       )
#     
#       select 
#         case 
#           when quarter = 1 then '1'
#           when quarter = 2 then '2'
#           when quarter = 3 then '3'
#           when quarter = 4 then '4'
#           else null end as quarter,
#         total_amount,
#         num_contributions,
#         total_donors,
#         avg_donation,
#         avg_contributions
#       from quarterly_fundraising_summary
#     
# """
# sql_query = jinja2.Template(raw_query).render(vars())

In [ ]:
pct_quarter = round((donations_quarterly["total_amount"].iloc[3] / total_2021) * 100, 2)

### 2021 Quarterly Breakdown

Our most successful fundraising quarter in 2021 was Quarter 4 (Graph 2), where {{pct_quarter}}% of 2021 donations were raised. Quarter 4 also had the most contributions and highest average donation size (Table 3). Quarter 2 had the lowest amount raised. 2020 followed a similar trend.

The most successful fundraising month was December, where almost a quarter ({{pct_month}}%) of 2021 donations were raised (Graph 3).

In [ ]:
import altair
chart_donations_quarterly = altair.Chart.from_json("""
{
    "width": 500,
    "height": 500,
    "layer": [
        {
            "data": {
                "name": "layer00"
            },
            "mark": {
                "clip": true,
                "type": "bar",
                "tooltip": true
            },
            "encoding": {
                "x": {
                    "type": "nominal",
                    "field": "quarter",
                    "title": "Quarter"
                },
                "y": {
                    "type": "quantitative",
                    "field": "total_amount",
                    "title": "Total Amount ($)"
                },
                "color": {
                    "value": "#654A94"
                }
            }
        }
    ],
    "$schema": "https://vega.github.io/schema/vega-lite/v4.json",
    "resolve": {
        "scale": {
            "y": "independent"
        }
    },
    "datasets": {
        "layer00": [
            {
                "name": "dummy",
                "value": 0
            }
        ]
    }
}
""")
chart_donations_quarterly.datasets.layer00 = donations_quarterly.to_json(orient='records')
chart_donations_quarterly.display(actions=False)

In [ ]:
donations_quarterly

In [ ]:
import altair
chart_donation_month = altair.Chart.from_json("""
{
    "width": 500,
    "height": 500,
    "layer": [
        {
            "data": {
                "name": "layer00"
            },
            "mark": {
                "clip": true,
                "type": "bar",
                "tooltip": true
            },
            "encoding": {
                "x": {
                    "type": "temporal",
                    "field": "month",
                    "title": "Month"
                },
                "y": {
                    "type": "quantitative",
                    "field": "total_amount",
                    "title": "Total Amount ($)"
                },
                "color": {
                    "value": "#D99E0B"
                }
            }
        }
    ],
    "config": {
        "legend": {
            "disable": false
        }
    },
    "$schema": "https://vega.github.io/schema/vega-lite/v4.json",
    "resolve": {
        "scale": {
            "y": "independent"
        }
    },
    "datasets": {
        "layer00": [
            {
                "name": "dummy",
                "value": 0
            }
        ]
    }
}
""")
chart_donation_month.datasets.layer00 = donation_month.to_json(orient='records')
chart_donation_month.display(actions=False)

In [ ]:
# import jinja2
# raw_query = """
#     with summary as (
#         select
#           date_part_year(paid_at::date) as year,
#           sum(amount) as total_amount,
#           count(donation_id) as num_contributions,
#           count(distinct email) as total_donors,
#           avg(amount) as avg_donation,
#           (num_contributions::decimal / total_donors::decimal)::float as avg_contributions
#         from sunrise_dbt.recurring_donations
#         where year in (2020,2021)
#         group by 1
#         order by 1
#       )
#     
#       select * from summary
# """
# sql_query = jinja2.Template(raw_query).render(vars())

In [ ]:
# import jinja2
# raw_query = """
#     with summary as (
#         select
#         distinct email,
#         date_part_year(created_at::date) as year
#         from sunrise_dbt.recurring_donations
#         where year = 2021
#       )
#     
#     select count(*) as new_recurring from summary
# """
# sql_query = jinja2.Template(raw_query).render(vars())

In [ ]:
# import jinja2
# raw_query = """
#     with summary as (
#         select
#         distinct email,
#         date_part_year(created_at::date) as year,
#       	date_trunc('month', created_at::date) as month
#         from sunrise_dbt.recurring_donations
#         where year = 2021
#       )
#     
#     select 
#     count(distinct email),
#     month
#     from summary
#     group by month
# """
# sql_query = jinja2.Template(raw_query).render(vars())

In [ ]:
recur_2021 = round(recurring["total_amount"].iloc[1], 2)
recur_2020 = round(recurring["total_amount"].iloc[0], 2)
pct_new = round(new_recurring["new_recurring"].iloc[0] / recurring["total_donors"].iloc[1] * 100, 2)
pct_donors = round(recurring["total_donors"].iloc[1] / donation_summary["total_donors"].iloc[1] * 100, 2)
pct_amount = round(recurring["total_amount"].iloc[1] / donation_summary["total_raised"].iloc[1] * 100, 2)

### Reccuring Donors

Recurring donors donated a total of ${{recur_2021}} in 2021, which is an increase of 20.68% compared to the previous year (Metric 2). In 2021, there were less recurring donors and more recurring contributions (Table 4). Donors stayed on longer as recurring contributors and on average were giving slightly larger gifts. 

In 2021, there were 9,726 recurring donors, and {{pct_new}}% of them were new to Sunrise. December, closely followed by January, was the month with the most new recurring donors (Graph 4). Almost half of all donors were recurring donors ({{pct_donors}}%) and recurring donations made up over half of the money raised ({{pct_amount}}%).
 

In [ ]:
recur_2021

In [ ]:
recur_2020

In [ ]:
recurring

In [ ]:
import altair
chart_monthly_new_reccuring = altair.Chart.from_json("""
{
    "width": 500,
    "height": 500,
    "$schema": "https://vega.github.io/schema/vega-lite/v4.json",
    "layer": [
        {
            "data": {
                "name": "layer00"
            },
            "mark": {
                "tooltip": true,
                "type": "bar",
                "clip": true
            },
            "encoding": {
                "x": {
                    "field": "month",
                    "type": "temporal",
                    "title": "Month"
                },
                "y": {
                    "field": "count",
                    "type": "quantitative",
                    "title": "# of New Recurring Donors"
                },
                "color": {
                    "value": "#654A94"
                }
            }
        }
    ],
    "resolve": {
        "scale": {}
    },
    "datasets": {
        "layer00": [
            {
                "name": "dummy",
                "value": 0
            }
        ]
    }
}
""")
chart_monthly_new_reccuring.datasets.layer00 = monthly_new_reccuring.to_json(orient='records')
chart_monthly_new_reccuring.display(actions=False)

In [ ]:
# import jinja2
# raw_query = """
#     with donations as (
#     
#       select * from sunrise_dbt.donations
#       where paid_at like '%2021%'
#     
#     ),
#     
#     donation_amount as (
#     
#     select
#       case
#        when amount between 0.01 and 49.99 then 'Under 50'
#        when amount between 50.00 and 99.99 then '50-99'
#        when amount between 100.00 and 499.99 then '100-499'
#        when amount between 500.00 and 999.99 then '500-999'
#        when amount between 1000.00 and 4999.99 then '1,000-4,999'
#        when amount between 5000.00 and 9999.99 then '5,000-9,999'
#        when amount between 10000.00 and 19999.99 then '10,000-19,999'
#        when amount between 20000.00 and 99999.99 then '20,000-99,999'
#        when amount between 100000.00 and 249999.99 then '100,000-249,999'
#        when amount between 250000.00 and 499999.99 then '250,000-499,999'
#        when amount between 500000.00 and 999999.99 then '500,000-999,999'
#        when amount between 1000000.00 and 4999999.99 then '1,000,000-4,999,999'
#        when amount <= 5000000.00 then '5,000,000+'
#        else null end as amount_bucket,
#       sum(amount) as amount
#     from donations
#     group by 1
#     ),
#     
#     donation_order as (
#       select * from donation_amount
#       order by
#         case when amount_bucket ='Under 50' then 1
#              when amount_bucket ='50-99' then 2
#              when amount_bucket ='100-499' then 3
#              when amount_bucket ='500-999' then 4
#              when amount_bucket ='1,000-4,999' then 5
#              when amount_bucket ='5,000-9,999' then 6
#              when amount_bucket ='10,000-19,999' then 7
#              when amount_bucket ='20,000-99,999' then 8
#              when amount_bucket ='100,000-249,999' then 9
#              when amount_bucket ='250,000-499,999' then 10
#              when amount_bucket ='500,000-999,999' then 11
#              when amount_bucket ='1,000,000-4,999,999' then 12
#              when amount_bucket ='5,000,000+' then 13
#              else null end
#       )
#     
#     select * from donation_order
# """
# sql_query = jinja2.Template(raw_query).render(vars())

In [ ]:
amount_small = round(donation_bucket["amount"].iloc[0], 2)
pct_small = round((amount_small / total_2021) * 100, 2)

### Small Dollar Donations 

Small dollar donations (under \$50) raised \${{amount_small}} (Graph 5), which was {{pct_small}}% of total donations in 2021. The second highest raising donation range was $100-499.

Of small dollar donations, the most money was raised in the \$25-50 range (Graph 6).

In [ ]:
# import jinja2
# raw_query = """
#     with donations as (
#     
#       select * from sunrise_dbt_ag.donations
#       where paid_at like '%2021%'
#     
#     ),
#     
#     donation_amount as (
#     
#     select
#       case
#        when amount between 0.01 and 9.99 then 'Under 10'
#        when amount between 10.00 and 24.99 then '10-24'
#        when amount between 25.00 and 50 then '25-50'
#        end as amount_bucket,
#       sum(amount) as amount
#     from donations
#     where amount_bucket is not null
#     group by 1
#     ),
#     
#     donation_order as (
#       select * from donation_amount
#       order by
#         case when amount_bucket ='Under 10' then 1
#              when amount_bucket ='10-24' then 2
#              when amount_bucket ='25-50' then 3
#             end
#       )
#     
#     select * from donation_order
# """
# sql_query = jinja2.Template(raw_query).render(vars())

In [ ]:
import altair
chart_donation_bucket = altair.Chart.from_json("""
{
    "width": 500,
    "height": 500,
    "layer": [
        {
            "data": {
                "name": "layer00"
            },
            "mark": {
                "clip": true,
                "type": "bar",
                "tooltip": true
            },
            "encoding": {
                "x": {
                    "sort": null,
                    "type": "nominal",
                    "field": "amount_bucket",
                    "title": "Amount Range"
                },
                "y": {
                    "type": "quantitative",
                    "field": "amount",
                    "title": "Total Amount ($)"
                },
                "color": {
                    "value": "#D99E0B"
                }
            }
        }
    ],
    "$schema": "https://vega.github.io/schema/vega-lite/v4.json",
    "resolve": {
        "scale": {
            "y": "independent"
        }
    },
    "datasets": {
        "layer00": [
            {
                "name": "dummy",
                "value": 0
            }
        ]
    }
}
""")
chart_donation_bucket.datasets.layer00 = donation_bucket.to_json(orient='records')
chart_donation_bucket.display(actions=False)

In [ ]:
import altair
chart_small_dollar = altair.Chart.from_json("""
{
    "width": 500,
    "height": 500,
    "layer": [
        {
            "data": {
                "name": "layer00"
            },
            "mark": {
                "clip": true,
                "type": "bar",
                "tooltip": true
            },
            "encoding": {
                "x": {
                    "sort": null,
                    "type": "nominal",
                    "field": "amount_bucket",
                    "title": "Amount Range"
                },
                "y": {
                    "type": "quantitative",
                    "field": "amount",
                    "title": "Total Amount ($)"
                },
                "color": {
                    "value": "#654A94"
                }
            }
        }
    ],
    "$schema": "https://vega.github.io/schema/vega-lite/v4.json",
    "resolve": {
        "scale": {
            "y": "independent"
        }
    },
    "datasets": {
        "layer00": [
            {
                "name": "dummy",
                "value": 0
            }
        ]
    }
}
""")
chart_small_dollar.datasets.layer00 = small_dollar.to_json(orient='records')
chart_small_dollar.display(actions=False)

In [ ]:
# import jinja2
# raw_query = """
#     select * 
#     from sunrise_dbt.donations
# """
# sql_query = jinja2.Template(raw_query).render(vars())

### Channel Breakdown

The website brought in the most money in 2021, as it did in 2020.

Note: Channels were defined based on the refcode. If a contribution was recurring, only the first instance was categorized under the channel that brought it in. For example, if someone donated through the website and committed to a recurring contribution, only the first contribution is included in the 'websiteâ channel. 

In [ ]:
# import jinja2
# raw_query = """
#     with
#     
#     donations as (
#       select * from sunrise_dbt.donations
#     ),
#     
#     recurring as (
#       select
#       *,
#       row_number() over(partition by unique_identifier order by paid_at::date asc) = 1 as is_first
#       from donations
#     ),
#     
#     first_donation as (
#       select * from recurring where is_first= 1
#     ),
#     
#     
#     donations_channel_2020 as (
#       select
#         date_trunc('year', paid_at) as year,
#         channel,
#         sum(amount) as total_amount_2020,
#         count(donation_id) as total_contributions_2020,
#         total_amount_2020 / total_contributions_2020 as avg_amount_2020
#       from first_donation
#       where year like '%2020%'
#       group by 1,2
#     ),
#     
#     donations_channel_2021 as (
#       select
#         date_trunc('year', paid_at) as year,
#         channel,
#         sum(amount) as total_amount_2021,
#         count(donation_id) as total_contributions_2021,
#         total_amount_2021 / total_contributions_2021 as avg_amount_2021
#       from first_donation
#       where year like '%2021%'
#       group by 1,2
#     )
#     
#     select 
#         a.channel,
#         total_amount_2020,
#         total_amount_2021,
#         (total_amount_2021 - total_amount_2020)/total_amount_2020 as amt_percent_difference,
#         total_contributions_2020,
#         total_contributions_2021,
#         ((total_contributions_2021 - total_contributions_2020)::decimal/total_contributions_2020::decimal)::float as contr_percent_difference,
#         avg_amount_2020,
#         avg_amount_2021,
#         (avg_amount_2021 - avg_amount_2020)/avg_amount_2020 as avg_percent_difference
#     from donations_channel_2021 as a
#     left join donations_channel_2020 as b
#     on a.channel = b.channel
#     order by 3 desc
# """
# sql_query = jinja2.Template(raw_query).render(vars())

In [ ]:
channel_breakdown

In [ ]:
# import jinja2
# raw_query = """
#     select * 
#     from sunrise_dbt.annual_donor_demographics
#     where year in (2020,2021)
#     order by year asc
# """
# sql_query = jinja2.Template(raw_query).render(vars())

In [ ]:
# import jinja2
# raw_query = """
#     select
#     year,
#     num_bipoc::int,
#     num_non_bipoc::int,
#     pct_bipoc::float,
#     pct_non_bipoc::float,
#     pct_with_race::float
#     from sunrise_dbt.annual_donor_demographics
#     where year in (2020,2021)
#     order by year asc
# """
# sql_query = jinja2.Template(raw_query).render(vars())

In [ ]:
# import jinja2
# raw_query = """
#     select
#     pct_white::float,
#     pct_black::float,
#     pct_asian::float,
#     pct_latinx::float,
#     pct_middle_eastern::float,
#     pct_native::float,
#     pct_native_hawaiin::float,
#     pct_pacific_islander::float,
#     pct_race_other::float
#     from sunrise_dbt.annual_donor_demographics
#     where year like '%2021%'
# """
# sql_query = jinja2.Template(raw_query).render(vars())

In [ ]:
# import jinja2
# raw_query = """
#     select
#     year,
#     num_working_class_summary::int,
#     num_not_working_class_summary::int,
#     pct_working_class_summary::float,
#     pct_not_working_class_summary::float,
#     pct_with_class::float
#     from sunrise_dbt.annual_donor_demographics
#     where year in (2020,2021)
#     order by year asc
# """
# sql_query = jinja2.Template(raw_query).render(vars())

In [ ]:
# import jinja2
# raw_query = """
#     select
#     pct_poor::float,
#     pct_working_class::float,
#     pct_middle_class::float,
#     pct_professional_class::float,
#     pct_upper_class::float
#     from sunrise_dbt.annual_donor_demographics
#     where year like '%2021%'
# """
# sql_query = jinja2.Template(raw_query).render(vars())

In [ ]:
# import jinja2
# raw_query = """
#     select
#     year,
#     num_gender_non_conforming::int,
#     num_cis::int,
#     pct_gender_non_conforming::float,
#     pct_cis::float,
#     pct_with_gender::float
#     from sunrise_dbt.annual_donor_demographics
#     where year in (2020,2021)
#     order by year asc
# """
# sql_query = jinja2.Template(raw_query).render(vars())

In [ ]:
# import jinja2
# raw_query = """
#     select
#     pct_women::float,
#     pct_men::float,
#     pct_trans::float,
#     pct_nonbinary::float,
#     pct_gender_other::float
#     from sunrise_dbt.annual_donor_demographics
#     where year like '%2021%'
# """
# sql_query = jinja2.Template(raw_query).render(vars())

In [ ]:
# import jinja2
# raw_query = """
#     with ages as (
#       select 
#         distinct email as email,
#         case when age < 20 then 'Under 20'
#              when age between 20 and 29 then '20-29'
#              when age between 30 and 39 then '30-39'
#              when age between 40 and 49 then '40-49'
#              when age between 50 and 59 then '50-59'
#              when age between 60 and 69 then '60-69'
#              when age between 70 and 79 then '70-79'
#              when age between 80 and 89 then '80-89'
#              when age between 90 and 99 then '90-99'
#              else null end as age_range
#       from sunrise_dbt.donors
#       where age < 100
#       and year = 2021
#     )
#     
#     select 
#     age_range,
#     count(email)
#     from ages
#     group by age_range
#     order by case when age_range = 'Under 20' then 1
#              when age_range = '20-29' then 2
#              when age_range = '30-39' then 3
#              when age_range = '40-49' then 4
#              when age_range = '50-59' then 5
#              when age_range = '60-69' then 6
#              when age_range = '70-79' then 7
#              when age_range = '80-89' then 8
#              when age_range = '90-99' then 9
#              else 10 end
# """
# sql_query = jinja2.Template(raw_query).render(vars())

In [ ]:
percent_with_race = round(demographics["pct_with_race"].iloc[1] * 100, 2)
percent_bipoc = round(demographics["pct_bipoc"].iloc[1] * 100, 2)

percent_with_class = round(demographics["pct_with_class"].iloc[1] * 100, 2)
percent_wc = round(demographics["pct_working_class_summary"].iloc[1] * 100, 2)

percent_with_gender = round(demographics["pct_with_gender"].iloc[1] * 100, 2)
percent_cis = round(demographics["pct_cis"].iloc[1] * 100, 2)

### Demographics

#### Race

Of the {{percent_with_race}}% of donors whose race information we have, the majority were white (Table 6). {{percent_bipoc}}% were BIPOC, a slight decrease from last year.

In [ ]:
race_compare

In [ ]:
race_breakdown

#### Class

Of the {{percent_with_class}}% of donors whose class information we have, the majority were middle class and up (Table 8). {{percent_wc}}% were working class (for this anaylsis, working class refers to anyone who identifies as poor or working class). In Table 9, we see that the highest percentage of donors we middle, followed closely by professional class.

In [ ]:
class_compare

In [ ]:
class_breakdown

#### Gender

Of the {{percent_with_gender}}% of donors whose class information we have, the majority were cis, {{percent_cis}}% (Table 10). In Table 11, we see that most donors were women.

In [ ]:
gender_compare

In [ ]:
gender_breakdown

#### Age

We have age data on 38.33% of our 2021 donors. The average age of our 2021 donors is 49 years old, compared to 48 years old in 2020. There were some outliers of people over 100 years old, so these averages have removed ages over 99.


In [ ]:
import altair
chart_query_result_3 = altair.Chart.from_json("""
{
    "width": 500,
    "height": 500,
    "layer": [
        {
            "data": {
                "name": "layer00"
            },
            "mark": {
                "clip": true,
                "type": "bar",
                "tooltip": true
            },
            "encoding": {
                "x": {
                    "sort": null,
                    "type": "nominal",
                    "field": "age_range",
                    "title": "Age Range"
                },
                "y": {
                    "type": "quantitative",
                    "field": "count",
                    "title": "Number of Donors"
                },
                "color": {
                    "value": "#D99E0B"
                }
            }
        }
    ],
    "$schema": "https://vega.github.io/schema/vega-lite/v4.json",
    "resolve": {
        "scale": {
            "y": "independent"
        }
    },
    "datasets": {
        "layer00": [
            {
                "name": "dummy",
                "value": 0
            }
        ]
    }
}
""")
chart_query_result_3.datasets.layer00 = query_result_3.to_json(orient='records')
chart_query_result_3.display(actions=False)